In [ ]:
using DifferentialEquations, LinearAlgebra, Plots, LsqFit

Versions [Pluto](https://github.com/vlc1/ene-4102c-td/blob/master/td4.jl) et [Jupyter](https://vlc1.github.io/ene-4102c/td4.ipynb) de ce notebook.


# Résolution du problème de Blasius

On se propose de résoudre le problème de Blasius de deux façons :

1. Par la méthode de tir (*shooting*), implémentée dans la bibliothèque `DifferentialEquations.jl`,
1. En exploitant une symétrie du problème.

Le problème s'écrit :
$$
y''' \left ( x \right ) + y \left ( x \right ) y'' \left ( x \right ) = 0
$$
où la variable dépendante $y$ vérifie les conditions aux limites
$$
\left \{ \begin{aligned}
y \left ( 0 \right ) & = 0, \\
y' \left ( 0 \right ) & = 0, \\
y' \left ( \infty \right ) & = 1.
\end{aligned} \right .
$$

L'une des difficultés rencontrées lors de la résolution numérique de ce problème est qu'il est défini sur un domaine semi-infini. On se propose ici d'utiliser la méthode dite *domain truncation*, qui consiste à résoudre l'équation sur un domaine fini mais suffisamment grand pour ne pas influencer la solution numérique, ce qui doit bien sûr être vérifié *a posteriori*.

## Problème aux limites et méthode de tir

1. Quel est le degré de cette EDO ? La réécrire sous la forme d'un système d'EDO d'ordre 1.
2. À partir de ce [tutoriel](https://diffeq.sciml.ai/stable/tutorials/bvp_example/), résoudre l'équation de Blasius sur le domaine $\left [ 0, 10000 \right ]$. On pourra utiliser les fonctions suivantes.

```julia
function blasius!(z, y, p, x)
    z[1] = y[2]
    z[2] = y[3]
    z[3] = -y[1] * y[3]
end
```
```julia
function bc!(r, y, p, x)
    r[1] = y[begin][1]
    r[2] = y[begin][2]
    r[3] = y[end][2] - 1
end
```

## Problème de Cauchy et symétrie

3. Montrer que l'équation de Blasius et les conditions en $0$ sont inchangées par la transformation
$$
\begin{aligned}
\overline{y} & \leftarrow c y, \\
\overline{x} & \leftarrow x / c.
\end{aligned}
$$

On suppose maintenant que $\overline{y}$ est soumis à la même équation que $y$, à l'exception de la condition en $\infty$ qui est remplacée par :
$$
\overline{y}'' \left ( 0 \right ) = 1.
$$
On définit alors
$$
\alpha = \overline{y}' \left ( \infty \right ).
$$

4. Montrer que le choix de la constante $c = \sqrt{\alpha}$ mène à
$$
y' \left ( \infty \right ) = 1.
$$
5. Résoudre $\overline{y}$ numériquement à partir de ce [tutoriel](https://diffeq.sciml.ai/stable/tutorials/ode_example/) et en déduire la solution de Blasius intégrant le problème numériquement une seule fois.

## Vérification

6. Vérifier qu'augmenter la taille du domaine d'intégration n'affecte pas la solution numérique.


# Conduction stationnaire dans une barre

On se propose de résoudre l'équation
$$
0 = y'' \left ( x \right ) + f \left ( x \right )
$$
avec les conditions
$$
\left \{ \begin{aligned}
y ' \left ( 0 \right ) & = g_0, \\
y \left ( 1 \right ) & = y_1.
\end{aligned} \right .
$$

$f$ représente une source (par exemple un dépot de chaleur par laser).

1. Soit $n$ le nombre de points. Quelle valeur de $h$ utiliser pour obtenir une discrétisation d'ordre 2 ? Implémenter la fonction `spacing` qui retourne $h$ en fonction de $n$, et enfin `mesh` qui retourne un tableau contenant les abscisses de chacune des inconnues.


In [ ]:
spacing(n) = 0.0

In [ ]:
mesh(n) = zeros(n)

2. Discrétiser l'équation, et l'implémenter à l'aide de deux fonctions, `laplacian` et `rhs`, invoquée depuis la fonction `numerical` ci-dessous.


In [ ]:
function laplacian(n)
    A = Tridiagonal(zeros.((n - 1, n, n - 1))...)

    for i in 1:n
        A[i, i] = 1.0
    end

    A
end

In [ ]:
rhs(n, f, g₀, y₁) = zeros(n)

In [ ]:
function numerical(n, f, g₀, y₁)
    A, B = laplacian(n), rhs(n, f, g₀, y₁)
    A \ B
end

3. Définir la solution analytique du problème suivant (fonction `analytical`) :


In [ ]:
begin
    f(x) = -one(x)
    g₀ = 0.0
    y₁ = 0.5
end

In [ ]:
analytical(x) = zero(x)

4. Résoudre le problème pour $n = 2$, $4$... et visualiser les solutions numérique et analytique. Que constatez-vous ?


In [ ]:
N = [2, 4, 8, 16]

In [ ]:
begin
    fig = plot()
    for n in N
        X, Y = mesh(n), numerical(n, f, g₀, y₁)
        scatter!(fig, X, Y)
        plot!(fig, X, analytical.(X))
    end
end

In [ ]:
fig

5. Construire un autre problème en modifiant `f`, `g₀`, `y₁` et `analytical` de sorte à que la solution ne soit plus un polynôme d'ordre 2.
6. Calculer les erreurs $L_1$, $L_2$ et $L_\infty$ pour plusieurs maillages et normes. Enfin, estimer l'ordre de convergence pour chacune des normes par la méthode des moindres carrés.
